<a href="https://colab.research.google.com/github/Seongjin1225/AI_SCHOOL_9/blob/main/ML%26DL/12%EC%9B%94%205%EC%9D%BC/12%EC%9B%94_5%EC%9D%BC_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression 구현
- Wisconsin breast cancer 데이터 셋을 사용할 거에요!

In [ ]:
# 필요 라이브러리 호출
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

from sklearn import linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
# Raw Data Loading
cancer = load_breast_cancer()

# x_data = cancer.data
# t_data = cancer.target
cancer.data.shape
cancer.target.shape

np.unique(cancer.target, return_counts=True)  # [212, 357]
# 약간의 데이터 불균형이 존재하는 것을 알 수 있다

# 데이터셋 저장
x_data = cancer.data
t_data = cancer.target

# 이상치 확인
# plt.boxplot(x_data)
# plt.show()

# 정규화 진행!
# 원칙적으로는 이상치 제거 후, 정규화 진행이 맞음
# but, boxplot상에서는 아싱치이더라도, 실제로는 이상치가 아닌 경우가 있다!!
# 여기서는, 실제적인 이상치는 존재하지 않는다고 가정하고 진행!
# t_data는 이미 0,1로만 구성되어 있어므로, x_data에 대해서만 진행
scaler = MinMaxScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

# 데이터를 분리해야 해요!
# 학습용과 테스트용으로 분리!(데이터를 섞어서 분리)
x_data_train_norm, x_data_test_norm, t_data_train, t_data_test = \
train_test_split(x_data_norm,
                 t_data,
                 stratify=t_data,  # 데이터의 비율에 맞춰서 나누라는 의미, 여기서는 t_data
                 test_size=0.2,  # 테스트 데이터셋의 비율 결정, default 값은 0.25
                 random_state=3,
                 )

# 섞어서 분리했기 때문에 데이터의 편향은 없을거라 판다
# 그래도 혹시모르니까 한번 확인해봅시다
np.unique(t_data_test,return_counts=True)

(array([0, 1]), array([42, 72]))

In [ ]:
# sklearn 구현
sklearn_model = linear_model.LogisticRegression()

# 학습하기 전, cross validation을 진행할 예정
# train data를 사용할 거에요!
score = cross_val_score(sklearn_model,
                        x_data_train_norm,
                        t_data_train,
                        cv=5)
score = np.mean(score)
score

# 학습 진행
sklearn_model.fit(x_data_train_norm,t_data_train)

# 모델 최종 평가
test_score = sklearn_model.score(x_data_test_norm,
                                 t_data_test)
test_score


0.9649122807017544

In [ ]:
# tensorflow 구현

keras_model = Sequential()

keras_model.add(Flatten(input_shape=(30,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))

keras_model.compile(optimizer=Adam(learning_rate = 1e-2),
                    loss='binary_crossentropy',
                    metrics=['acc'])  # 정확도 확인을 위해

# 학습하기
keras_model.fit(x_data_train_norm,
                t_data_train,
                epochs=300,
                verbose=1,
                validation_split=0.2)  # 학습 데이터의 20프로를 테스트 데이터로 사용 의미
                                       # 각 epoch마다 검증 진행

# loss: 0.0789 - acc: 0.9808 - val_loss: 0.0935 - val_acc: 0.9560 > 학습 후 마지막 결과
# loss, acc는 별 의미가 없다! why? -> 학습 데이터를 가지고 진행하는 거라
# training_data로 평가 vs. validation_data로 평가

In [ ]:
# 학습이 다 끝났으므로
# evaluation(평가)
result = keras_model.evaluate(x_data_test_norm,
                     t_data_test)
result

# [0.0907793641090393, 0.9736841917037964]
# loss값, acc값

4/4 [==============================] - 0s 5ms/step - loss: 0.0908 - acc: 0.9737


[0.0907793641090393, 0.9736841917037964]

3 # Titanic Dataset을 활용한 머신러닝 학습
- Kaggle에 올라온 데이터를 가지고 진행
- Kaggle에 있는 데이터를 전처리 한 후
- 모델을 만들어서 학습을 진행
- 자체 평가를 진행(validation)
- 모델을 이용해서 예측값을 추출(test.csv)
- 예측된 결과를 kaggle에 upload해서 우리 모델의 성능을 검증.


In [ ]:
# 필요 라이브러리 호출
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Raw Data Loading
df = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/train.csv')
df

# 데이터 전처리
train = df

# 사용하는 컬럼만 남겨두기, 필요없는 컬럼은 제거
# 필요없는 컬럼 -> 종속변수에 영향을 주지 않거나, 의미상 중복되는 값
train.drop(['PassengerId','Name','Ticket','Cabin','Fare'],axis=1,inplace=True)
df

# 성별 컬럼 0,1 숫자로 표시
# map 함수 사용하면 편리하게 가능
gender_mapping = {'male':0,
                  'female':1}
train['Sex'] = train['Sex'].map(gender_mapping)
train['Family'] = train['SibSp'] + train['Parch']
train.drop(['SibSp','Parch'],axis=1,inplace=True)
df

# Embarked의 결측치를 처리해 보아요!
# 편의상 Q로 대체
train['Embarked'] = train['Embarked'].fillna('Q')

# Embarked 처리
Embarked_mapping = {'S':0,
                    'C':1,
                    'Q':2}
train['Embarked'] = train['Embarked'].map(Embarked_mapping)
df

# 나이 컬럼의 결측치 처리 - 전체 나이의 평균으로
train['Age'] = train['Age'].fillna(train['Age'].mean())

# 나이 컬럼에 대해서는 binning을 해줄거에요!
# binning == 구간화
train.loc[train['Age'] < 8, 'Age'] = 0
train.loc[(train['Age'] >= 8) & (train['Age'] < 20),'Age'] = 1
train.loc[(train['Age'] >= 20) & (train['Age'] < 65),'Age'] = 2
train.loc[train['Age'] >= 65, 'Age'] = 3
df

# 데이터셋 지정
x_data = train.drop('Survived',axis=1,inplace=False)
t_data = train['Survived'].values.reshape(-1,1)

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

# 이번 예제의 경우 test 데이터가 따로 존재하므로
# 만들 필요 없음

In [ ]:
# 모델 구현
keras_model = Sequential()

keras_model.add(Flatten(input_shape=(5,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))

keras_model.compile(optimizer=Adam(learning_rate=1e-2),
                    loss='binary_crossentropy',
                    metrics=['acc'])

keras_model.fit(x_data_norm,
                t_data,
                epochs=300,
                verbose=1,
                validation_split=0.2)

In [ ]:
# 학습이 끝났으니 제출 파일 생성
# 예측 결과 저장

test = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/gender_submission.csv')

# 우선, test data 전처리 필요
test.drop(['PassengerId', 'Ticket', 'Name', 'Fare', 'Cabin'],axis=1,inplace=True)
gender_mapping = {'male':0,
                  'female':1}
test['Sex']= test['Sex'].map(gender_mapping)
test['Family'] = test['SibSp'] + test['Parch']
test.drop(['SibSp','Parch'],axis=1,inplace=True)

test['Embarked'] = test['Embarked'].fillna('Q')
Embarked_mapping = {'S': 0,
                    'C': 1,
                    'Q': 2}
test['Embarked'] = test['Embarked'].map(Embarked_mapping)

test['Age'].fillna(test['Age'].mean(),inplace=True)
test.loc[test['Age'] < 8, 'Age'] = 0
test.loc[(test['Age'] >= 8) & (test['Age'] < 20), 'Age'] = 1
test.loc[(test['Age'] >= 20) & (test['Age'] < 65), 'Age'] = 2
test.loc[test['Age'] >= 65, 'Age'] = 3

x_data_test_norm = scaler.transform(test.values)

predict = keras_model.predict(x_data_test_norm)
predict  # 확률의 형태로 출력됨

submission['Survived'] = predict
submission['Survived'] = np.where((submission['Survived'] >= 0.5), 1, 0)

submission

14/14 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
